# Named Entity Recognition (NER)

I will attempt to do the Named Entity Recognition task. I will train two models and demonstrate their results below.

The models are:<br>
1- spaCy NER Gold Parse system [1]. spaCy is an industrial strength NLP library that is widely used by large corporation to handle similar NLP applications. E.g. it's being used by BBC, Microsoft, and many other big companies.

2- Flair embeddings [2]. Contextual String Embeddings for Sequence Labeling is currently the state-of-the-art [3] system in Named Enitiy Recognition task, and the only system outperforming Google's BERT [4] model. More information on Flair can be found on their paper [5].

I have selected these two models to demonstrate that I am capable of providing a quick & relaiable solution when needed (spaCy). Also, when time/resource allows, I am capable of providing a siginfacntly better solution that is considered the state-of-the-art in the field of NLP (Flair embeddings).

spaCy needs a couple of hours to train a decent model on a potato laptop, while Flair embeddings (with CharLMEmbeddings) can take a couple of days on a proper workstation with GPUs. 

Requirements to run this code:
- python 3.6
- spacy '2.0.16'
- flair

[1] https://spacy.io/<br>
[2] https://github.com/zalandoresearch/flair<br>
[3] https://github.com/zalandoresearch/flair#comparison-with-state-of-the-art<br>
[4] https://ai.googleblog.com/2018/11/open-sourcing-bert-state-of-art-pre.html<br>
[5] https://drive.google.com/file/d/17yVpFA7MmXaQFTe-HDpZuqw9fJlmzg56/view)

## Initialization

In [1]:
import spacy
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy import displacy
import json
import random
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.trainers import SequenceTaggerTrainer
from flair.data import TaggedCorpus
from flair.data_fetcher import NLPTaskDataFetcher, NLPTask
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, CharLMEmbeddings, CharacterEmbeddings
from typing import List

## Training, Validation, Testing

In here we divide the provided training data into 3 parts; training (80%), validation (10%) and testing (10%).

In [2]:
# reading the raw data
raw_data_PATH = 'Dataset/'
File_ = open(raw_data_PATH+'ner_dataset.txt')

DATA = []
sentence = []

for line in File_:
    try:
        if line == '\n':
            DATA.append(sentence)
            sentence = []
        else:
            sentence.append(line)
    except:
        print('you have a bad line..',line)

random.shuffle(DATA)    # shuffling the data is always good to preven overfitting

# dividing the data into trainig (80%), validation (10%) and testing (10%).
split_ = int(0.1 * len(DATA))
TRAIN_DATA, VAL_DATA, TEST_DATA = DATA[:8*split_], DATA[8*split_:9*split_], DATA[9*split_:]
print('  *Training has (',len(TRAIN_DATA),') instances.')
print('  *Validation has (',len(VAL_DATA),') instances.')
print('  *TEST has (',len(TEST_DATA),') instances.')

# print()
# storing the data
for name,data in zip(['training','validation','test'],[TRAIN_DATA, VAL_DATA, TEST_DATA]):
    F = open(raw_data_PATH+'ner_dataset_'+name+'.txt','w')
    F.write('\n'.join([''.join(x) for x in TRAIN_DATA]))
    F.close()


  *Training has ( 14760 ) instances.
  *Validation has ( 1845 ) instances.
  *TEST has ( 1849 ) instances.


## 1- spaCy NER

### Preparing the data for spacy

Inspecting the data, shuffeling it and preparing it to be fed into the Spacy model.

In [3]:
# theses functions create training data suitable for the Spacy tool
def _reformat_data(data):
    for counter, example_ in enumerate(data):
        index_ = 0
        annotations = {}
        sentence, ner_tag = example_
        for word, tag in zip(sentence, ner_tag):
            #-------------------------------------#
            # analysing the NER tag
            if '-' in tag:
                In, tag = tag.split('-')
                if tag not in annotations:
                    annotations[tag] = []
            else:
                In = tag
                
            #-------------------------------------#
            # creating the training data
            if In == 'B':
                annotations[tag].append([index_, index_+len(word)])
            elif In == 'I':
                annotations[tag][-1][1] = index_+len(word)
            elif In != 'O':
                print('=====!!!!!', In)
                
            index_ += len(word) + 1
        
        # fix the format
        ann = {'entities':[ (val[0],val[1],key) for key in annotations for val in annotations[key]]}
            
        ## update the training data to fit spacy format
        text = ' '.join(sentence)
        data[counter] = (text, ann)
    return data

def _create_training_data(raw_data):
    File_ = open(raw_data, 'r')
    TRAIN_DATA = []
    sentence = []
    ner_tag = []

    for line in File_:
        try:
            line = line.split('\n')[0]

            if line == '':
                TRAIN_DATA.append([sentence,ner_tag])
                sentence = []
                ner_tag = []
            else:
                word, POS1, CNK2, tag = line.split(' ')
                sentence.append(word)
                ner_tag.append(tag)
        except:
            print('you have a bad line..',line)
            
    File_.close()
    return _reformat_data(TRAIN_DATA)
            

In [4]:
# reading the raw data
raw_data_PATH = 'Dataset/'

# prepare the training data into spacy format
TRAIN_DATA = _create_training_data(raw_data_PATH+'ner_dataset_training.txt')
VAL_DATA = _create_training_data(raw_data_PATH+'ner_dataset_validation.txt')
TEST_DATA = _create_training_data(raw_data_PATH+'ner_dataset_test.txt')

# Inspecting the data        
print('We have a total of',len(TRAIN_DATA),'training instances.')

# print class analysis
all_tags = [ent[2] for data in TRAIN_DATA for ent in data[1]['entities']]
classes = set(all_tags)
print('We have',len(classes),'classes in this dataset.',classes)

for c_ in classes:
    print('  -class(',c_,') has',all_tags.count(c_),'instances.')


We have a total of 14759 training instances.
We have 4 classes in this dataset. {'LOC', 'MISC', 'PER', 'ORG'}
  -class( LOC ) has 7171 instances.
  -class( MISC ) has 3478 instances.
  -class( PER ) has 6852 instances.
  -class( ORG ) has 6126 instances.


### Training the model

In [5]:
# Load or create a blank English model
model = 'Spacy/'
output_dir = 'Spacy/'

if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
        

"""Load the model, set up the pipeline and train the entity recognizer."""
if model is not None:
    try:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    except:
        nlp = spacy.blank('en')  # create blank Language class
        print("Could not find the model, Created a blank 'en' model isntead")
else:
    nlp = spacy.blank('en')  # create blank Language class
    print("Created blank 'en' model")
    

Could not find the model, Created a blank 'en' model isntead


In [6]:
def predict_on_texts(texts):
    colors = {}
    colors['ORG'] = 'orange'
    colors['PER'] = '#aa9cfc'
    colors['LOC'] = 'green'
    colors['MISC'] = 'yellow'
    options = {'ents': classes, 'colors': colors}

    for text in texts:
        doc = nlp(text)
        Entities = [(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]
        if len(Entities) > 0:
            displacy.render(doc, style='ent', jupyter=True, options=options)
        else:
            print('no entities detected: ',text)
        print('--------------------------')
        print()
    

In [7]:
def predict_on_test_set(filepath):
    ext = filepath.split('.')[-1]
    if ext == 'txt':
        VAL_DATA = _create_training_data(filepath) 
    else:
        VAL_DATA = []

    TP, FN, FP = 0, 0, 0 # True positives, False negatives, False Positives
    for text, ann in VAL_DATA:
        doc = nlp(text)
        GT = sorted(ann['entities'], key=lambda tup: tup[0])
        Entities = [(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]
        Ground_Truth = [(text[a[0]:a[1]], a[0], a[1], a[2]) for a in GT]
        
        TP += len([value for value in Entities if value in Ground_Truth])
        FP += len([value for value in Entities if value not in Ground_Truth])
        FN += len([value for value in Ground_Truth if value not in Entities])
    Pr, Re = TP/(TP+FP), TP/(TP+FN) ## computing Precision and Recall
    print('  -Validation: -precision=%.3f -recall=%.3f -f1 score=%.3f'  % (Pr, Re, 2*(Pr*Re)/(Pr+Re)))
        

In [8]:
# parameters
n_iter = 20 # number of iteration
                
# create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = nlp.get_pipe('ner')
    
# add labels to model
for ent in classes:
    ner.add_label(ent)

In [9]:
# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    print('*started trianing..')
    for itn in range(n_iter):
        # shuffle the data (reduce the overfitting of the model)
        random.shuffle(TRAIN_DATA)
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(TRAIN_DATA, size=compounding(4., 32., 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(
                texts,  # batch of texts
                annotations,  # batch of annotations
                drop=0.5,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
        ## printing training and validation losses
        print('  -Trainnig loss', losses)
        predict_on_test_set(raw_data_PATH+'ner_dataset_validation.txt')
        
        # save model to output directory
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)



*started trianing..
  -Trainnig loss {'ner': 535.5363936112482}
  -Validation: -precision=0.752 -recall=0.783 -f1 score=0.768
Saved model to Spacy
  -Trainnig loss {'ner': 187.14779767257355}
  -Validation: -precision=0.882 -recall=0.897 -f1 score=0.890
Saved model to Spacy
  -Trainnig loss {'ner': 134.02237596263717}
  -Validation: -precision=0.925 -recall=0.937 -f1 score=0.930
Saved model to Spacy
  -Trainnig loss {'ner': 108.67592926707505}
  -Validation: -precision=0.938 -recall=0.949 -f1 score=0.943
Saved model to Spacy
  -Trainnig loss {'ner': 84.78177396119163}
  -Validation: -precision=0.946 -recall=0.952 -f1 score=0.949
Saved model to Spacy
  -Trainnig loss {'ner': 78.67414079785877}
  -Validation: -precision=0.956 -recall=0.964 -f1 score=0.960
Saved model to Spacy
  -Trainnig loss {'ner': 70.96298747111538}
  -Validation: -precision=0.966 -recall=0.971 -f1 score=0.969
Saved model to Spacy
  -Trainnig loss {'ner': 62.735257567765636}
  -Validation: -precision=0.967 -recall=0.9

#### Testing the model

In [10]:
# To test the model on a txt file use:
# predict_on_test_set(raw_data_PATH+'ner_dataset.txt')

# To test the model with a sequence of sentences use:
predict_on_texts(['New York city','My name is Muhannad, and I live in the US. I work in Rolls Royce.'])


--------------------------



--------------------------



In [11]:
# Plotting the NER tags.
# using display-spacy to show the ner values (showing a sample only).
random_examples = [int(random.random()*len(VAL_DATA)) for i in range(4)]
texts = [VAL_DATA[ex][0] for ex in random_examples]
predict_on_texts(texts)

no entities detected:  and we will deal with such designs sternly , " the prime minister said .
--------------------------



--------------------------



--------------------------



--------------------------



## Flair model

Flair includes word embeddings to predict the named entites within the text.

In [12]:
# define columns
columns = {0: 'text', 1: 'pos', 2: 'cnk', 3:'ner'}

# this is the folder in which train, test and dev files reside
data_folder = 'Dataset/'

# training folder
flair_folder = 'Flair/'
flair_path = Path(flair_folder)
if not flair_path.exists():
    flair_path.mkdir()
        
# retrieve corpus using column format, data folder and the names of the train, dev and test files
# 1. get the corpus
corpus: TaggedCorpus = NLPTaskDataFetcher.fetch_column_corpus(data_folder, columns,
                                                              train_file='ner_dataset_training.txt',
                                                              test_file='ner_dataset_test.txt',
                                                              dev_file='ner_dataset_validation.txt')
                
# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary.idx2item)

# 4. initialize embeddings
embedding_types: List[TokenEmbeddings] = [

    # use this if you have a potato PC
    WordEmbeddings('glove'),

    # comment in this line to use character embeddings
    # CharacterEmbeddings(),

    # comment in these lines to use contextual string embeddings
#     CharLMEmbeddings('news-forward'),
#     CharLMEmbeddings('news-backward'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)
    

# 5. initialize sequence tagger
tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)
    

# 6. initialize trainer
trainer: SequenceTaggerTrainer = SequenceTaggerTrainer(tagger, corpus)

# 7. start training
trainer.train(flair_folder,
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=100)



[b'<unk>', b'O', b'B-PER', b'I-PER', b'B-LOC', b'B-ORG', b'I-ORG', b'B-MISC', b'I-MISC', b'I-LOC', b'<START>', b'<STOP>']
2018-11-29 01:00:01,795 Evaluation method: F1
2018-11-29 01:00:01,798 ----------------------------------------------------------------------------------------------------
2018-11-29 01:00:02,021 epoch 1 - iter 0/462 - loss 34.05309677
2018-11-29 01:00:12,215 epoch 1 - iter 46/462 - loss 10.13411792
2018-11-29 01:00:23,134 epoch 1 - iter 92/462 - loss 8.46139271
2018-11-29 01:00:34,241 epoch 1 - iter 138/462 - loss 7.50607529
2018-11-29 01:00:46,656 epoch 1 - iter 184/462 - loss 6.87369085
2018-11-29 01:00:57,415 epoch 1 - iter 230/462 - loss 6.40932856
2018-11-29 01:01:08,262 epoch 1 - iter 276/462 - loss 6.07861754
2018-11-29 01:01:19,153 epoch 1 - iter 322/462 - loss 5.81194002
2018-11-29 01:01:30,470 epoch 1 - iter 368/462 - loss 5.53879106
2018-11-29 01:01:41,213 epoch 1 - iter 414/462 - loss 5.33879499
2018-11-29 01:01:52,256 epoch 1 - iter 460/462 - loss 5.155

2018-11-29 01:19:37,688 epoch 7 - iter 460/462 - loss 2.15468833
2018-11-29 01:19:37,806 ----------------------------------------------------------------------------------------------------
2018-11-29 01:20:55,205 EPOCH 7: lr 0.1000 - bad epochs 1
2018-11-29 01:20:55,206 DEV : f-score 0.8705 - acc 0.8705 - tp 19779 - fp 2037 - fn 3848 - tn 19779
2018-11-29 01:20:55,206 TEST: f-score 0.8705 - acc 0.8705 - tp 19779 - fp 2037 - fn 3848 - tn 19779
2018-11-29 01:20:55,207 ----------------------------------------------------------------------------------------------------
2018-11-29 01:20:55,471 epoch 8 - iter 0/462 - loss 1.42024136
2018-11-29 01:21:05,252 epoch 8 - iter 46/462 - loss 1.96299510
2018-11-29 01:21:14,765 epoch 8 - iter 92/462 - loss 2.07168673
2018-11-29 01:21:24,358 epoch 8 - iter 138/462 - loss 2.14242340
2018-11-29 01:21:33,943 epoch 8 - iter 184/462 - loss 2.11860722
2018-11-29 01:21:42,894 epoch 8 - iter 230/462 - loss 2.08180723
2018-11-29 01:21:53,323 epoch 8 - iter 27

2018-11-29 01:39:30,639 epoch 14 - iter 184/462 - loss 1.82201212
2018-11-29 01:39:39,988 epoch 14 - iter 230/462 - loss 1.82177077
2018-11-29 01:39:49,721 epoch 14 - iter 276/462 - loss 1.83609324
2018-11-29 01:39:59,449 epoch 14 - iter 322/462 - loss 1.85219731
2018-11-29 01:40:08,723 epoch 14 - iter 368/462 - loss 1.83529518
2018-11-29 01:40:18,252 epoch 14 - iter 414/462 - loss 1.82956879
2018-11-29 01:40:28,860 epoch 14 - iter 460/462 - loss 1.81510421
2018-11-29 01:40:28,978 ----------------------------------------------------------------------------------------------------
2018-11-29 01:41:47,146 EPOCH 14: lr 0.1000 - bad epochs 0
2018-11-29 01:41:47,147 DEV : f-score 0.8966 - acc 0.8967 - tp 20721 - fp 1870 - fn 2906 - tn 20721
2018-11-29 01:41:47,148 TEST: f-score 0.8966 - acc 0.8967 - tp 20721 - fp 1870 - fn 2906 - tn 20721
2018-11-29 01:41:47,148 ----------------------------------------------------------------------------------------------------
2018-11-29 01:41:47,408 epoch

2018-11-29 01:59:51,905 ----------------------------------------------------------------------------------------------------
2018-11-29 01:59:52,112 epoch 21 - iter 0/462 - loss 0.93344438
2018-11-29 02:00:02,936 epoch 21 - iter 46/462 - loss 1.64582259
2018-11-29 02:00:13,239 epoch 21 - iter 92/462 - loss 1.70821893
2018-11-29 02:00:23,949 epoch 21 - iter 138/462 - loss 1.70152836
2018-11-29 02:00:33,635 epoch 21 - iter 184/462 - loss 1.68350503
2018-11-29 02:00:43,065 epoch 21 - iter 230/462 - loss 1.71820870
2018-11-29 02:00:52,619 epoch 21 - iter 276/462 - loss 1.72387379
2018-11-29 02:01:02,036 epoch 21 - iter 322/462 - loss 1.69021509
2018-11-29 02:01:11,485 epoch 21 - iter 368/462 - loss 1.68793401
2018-11-29 02:01:21,011 epoch 21 - iter 414/462 - loss 1.68958470
2018-11-29 02:01:29,725 epoch 21 - iter 460/462 - loss 1.67533234
2018-11-29 02:01:29,838 ----------------------------------------------------------------------------------------------------
2018-11-29 02:02:47,600 EPOC

2018-11-29 02:19:28,855 ----------------------------------------------------------------------------------------------------
2018-11-29 02:20:46,226 EPOCH 27: lr 0.1000 - bad epochs 1
2018-11-29 02:20:46,227 DEV : f-score 0.9180 - acc 0.9180 - tp 21273 - fp 1445 - fn 2354 - tn 21273
2018-11-29 02:20:46,227 TEST: f-score 0.9180 - acc 0.9180 - tp 21273 - fp 1445 - fn 2354 - tn 21273
2018-11-29 02:20:46,228 ----------------------------------------------------------------------------------------------------
2018-11-29 02:20:46,428 epoch 28 - iter 0/462 - loss 1.72887146
2018-11-29 02:20:55,741 epoch 28 - iter 46/462 - loss 1.50692750
2018-11-29 02:21:05,020 epoch 28 - iter 92/462 - loss 1.47013050
2018-11-29 02:21:14,531 epoch 28 - iter 138/462 - loss 1.51538355
2018-11-29 02:21:23,776 epoch 28 - iter 184/462 - loss 1.54053133
2018-11-29 02:21:33,482 epoch 28 - iter 230/462 - loss 1.55179512
2018-11-29 02:21:43,126 epoch 28 - iter 276/462 - loss 1.56868326
2018-11-29 02:21:52,682 epoch 28 

...

2018-11-29 04:37:20,894 epoch 74 - iter 322/462 - loss 1.28054260
2018-11-29 04:37:30,198 epoch 74 - iter 368/462 - loss 1.28022343
2018-11-29 04:37:39,583 epoch 74 - iter 414/462 - loss 1.27641652
2018-11-29 04:37:48,687 epoch 74 - iter 460/462 - loss 1.27451729
2018-11-29 04:37:48,842 ----------------------------------------------------------------------------------------------------
2018-11-29 04:39:06,306 EPOCH 74: lr 0.0500 - bad epochs 0
2018-11-29 04:39:06,307 DEV : f-score 0.9454 - acc 0.9454 - tp 22061 - fp 984 - fn 1566 - tn 22061
2018-11-29 04:39:06,308 TEST: f-score 0.9454 - acc 0.9454 - tp 22061 - fp 984 - fn 1566 - tn 22061
2018-11-29 04:39:06,309 ----------------------------------------------------------------------------------------------------
2018-11-29 04:39:06,557 epoch 75 - iter 0/462 - loss 0.86046386
2018-11-29 04:39:16,309 epoch 75 - iter 46/462 - loss 1.32381204
2018-11-29 04:39:25,653 epoch 75 - iter 92/462 - loss 1.26944749
2018-11-29 04:39:34,604 epoch 75 - 

2018-11-29 04:56:35,623 epoch 81 - iter 46/462 - loss 1.28110327
2018-11-29 04:56:44,991 epoch 81 - iter 92/462 - loss 1.21895466
2018-11-29 04:56:54,579 epoch 81 - iter 138/462 - loss 1.16673914
2018-11-29 04:57:04,266 epoch 81 - iter 184/462 - loss 1.18738899
2018-11-29 04:57:13,661 epoch 81 - iter 230/462 - loss 1.19983054
2018-11-29 04:57:23,145 epoch 81 - iter 276/462 - loss 1.22247770
2018-11-29 04:57:32,250 epoch 81 - iter 322/462 - loss 1.22041062
2018-11-29 04:57:41,611 epoch 81 - iter 368/462 - loss 1.22238994
2018-11-29 04:57:51,208 epoch 81 - iter 414/462 - loss 1.22901260
2018-11-29 04:58:00,936 epoch 81 - iter 460/462 - loss 1.22895486
2018-11-29 04:58:01,051 ----------------------------------------------------------------------------------------------------
2018-11-29 04:59:18,442 EPOCH 81: lr 0.0500 - bad epochs 0
2018-11-29 04:59:18,443 DEV : f-score 0.9504 - acc 0.9504 - tp 22210 - fp 902 - fn 1417 - tn 22210
2018-11-29 04:59:18,443 TEST: f-score 0.9504 - acc 0.9504 -

2018-11-29 05:16:52,042 DEV : f-score 0.9514 - acc 0.9514 - tp 22236 - fp 880 - fn 1391 - tn 22236
2018-11-29 05:16:52,043 TEST: f-score 0.9514 - acc 0.9514 - tp 22236 - fp 880 - fn 1391 - tn 22236
2018-11-29 05:16:52,044 ----------------------------------------------------------------------------------------------------
2018-11-29 05:16:52,257 epoch 88 - iter 0/462 - loss 1.43969154
2018-11-29 05:17:03,201 epoch 88 - iter 46/462 - loss 1.29068403
2018-11-29 05:17:13,507 epoch 88 - iter 92/462 - loss 1.18027021
2018-11-29 05:17:23,881 epoch 88 - iter 138/462 - loss 1.15806419
2018-11-29 05:17:33,349 epoch 88 - iter 184/462 - loss 1.14222054
2018-11-29 05:17:42,730 epoch 88 - iter 230/462 - loss 1.17642743
2018-11-29 05:17:52,717 epoch 88 - iter 276/462 - loss 1.17660003
2018-11-29 05:18:03,559 epoch 88 - iter 322/462 - loss 1.18822776
2018-11-29 05:18:14,714 epoch 88 - iter 368/462 - loss 1.18054102
2018-11-29 05:18:25,758 epoch 88 - iter 414/462 - loss 1.18000932
2018-11-29 05:18:36,7

2018-11-29 05:36:12,599 epoch 94 - iter 368/462 - loss 1.16127880
2018-11-29 05:36:21,776 epoch 94 - iter 414/462 - loss 1.16053764
2018-11-29 05:36:31,029 epoch 94 - iter 460/462 - loss 1.16312396
2018-11-29 05:36:31,136 ----------------------------------------------------------------------------------------------------
2018-11-29 05:37:48,731 EPOCH 94: lr 0.0500 - bad epochs 1
2018-11-29 05:37:48,732 DEV : f-score 0.9537 - acc 0.9537 - tp 22305 - fp 845 - fn 1322 - tn 22305
2018-11-29 05:37:48,733 TEST: f-score 0.9537 - acc 0.9537 - tp 22305 - fp 845 - fn 1322 - tn 22305
2018-11-29 05:37:48,734 ----------------------------------------------------------------------------------------------------
2018-11-29 05:37:48,981 epoch 95 - iter 0/462 - loss 1.67304444
2018-11-29 05:38:00,178 epoch 95 - iter 46/462 - loss 1.14772475
2018-11-29 05:38:10,855 epoch 95 - iter 92/462 - loss 1.16586770
2018-11-29 05:38:20,578 epoch 95 - iter 138/462 - loss 1.17999476
2018-11-29 05:38:29,950 epoch 95 - 

2018-11-29 05:55:29,484 MISC: f-score 0.9132 - acc 0.9132 - tp 3067 - fp 172 - fn 411 - tn 3067
2018-11-29 05:55:29,485 ORG : f-score 0.9276 - acc 0.9276 - tp 5610 - fp 360 - fn 516 - tn 5610
2018-11-29 05:55:29,492 PER : f-score 0.9785 - acc 0.9786 - tp 6692 - fp 133 - fn 160 - tn 6692
